In [2]:
# Build app using GenAI App (Take a doc, read / scrape after with help of LLMs and Prompts we should able to ask question and get answer from that APP. )
# Using this Web page https://docs.langchain.com/langsmith/home 
# Beautilfulsoup4 ( scrape the website datas.)
# Simple GEN AI APP Using LANGCHAIN + LLM models.  

import os 
from dotenv import load_dotenv
load_dotenv()

os.environ["OPENAI_API_KEY"]=os.getenv("OPENAI_API_KEY")
#Used for Langsmith tracking
os.environ["LANGCHAIN_API_KEY"]=os.getenv("LANGCHAIN_API_KEY")
os.environ["LANGCHAIN_TRACING_V2"]="true"
os.environ["LANGCHAIN_PROJECT"]=os.getenv("LANGCHAIN_PROJECT")



In [3]:
## Date Ingestion - From the website we need to scrape the data.

from langchain_community.document_loaders import WebBaseLoader

#Scrape the data.https://python.langchain.com/docs/tutorials/
loader=WebBaseLoader("https://python.langchain.com/docs/tutorials/")
docs=loader.load()
docs

USER_AGENT environment variable not set, consider setting it to identify your requests.


[Document(metadata={'source': 'https://python.langchain.com/docs/tutorials/', 'title': 'Tutorials | 🦜️🔗 LangChain', 'description': 'New to LangChain or LLM app development in general? Read this material to quickly get up and running building your first applications.', 'language': 'en'}, page_content='\n\n\n\n\nTutorials | 🦜️🔗 LangChain\n\n\n\n\n\n\n\n\nSkip to main contentThese docs will be deprecated and no longer maintained with the release of LangChain v1.0 in October 2025. Visit the v1.0 alpha docsIntegrationsAPI ReferenceMoreContributingPeopleError referenceLangSmithLangGraphLangChain HubLangChain JS/TSv0.3v0.3v0.2v0.1💬SearchIntroductionTutorialsBuild a Question Answering application over a Graph DatabaseTutorialsBuild a simple LLM application with chat models and prompt templatesBuild a ChatbotBuild a Retrieval Augmented Generation (RAG) App: Part 2Build an Extraction ChainBuild an AgentTaggingBuild a Retrieval Augmented Generation (RAG) App: Part 1Build a semantic search engineB

In [5]:
# Divide Documents to Chunks of text to LLMs (reason why chunks, LLm there is some limitations will context size).
# Load docs > divide Text into chunks > convert text/chunks to vectors (Using vectors embeddings eg OpenAI)>> store in Vectorstore DB.

from langchain_text_splitters import RecursiveCharacterTextSplitter
#split text to chunk
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
documents = text_splitter.split_documents(docs)
documents



[Document(metadata={'source': 'https://python.langchain.com/docs/tutorials/', 'title': 'Tutorials | 🦜️🔗 LangChain', 'description': 'New to LangChain or LLM app development in general? Read this material to quickly get up and running building your first applications.', 'language': 'en'}, page_content='Tutorials | 🦜️🔗 LangChain'),
 Document(metadata={'source': 'https://python.langchain.com/docs/tutorials/', 'title': 'Tutorials | 🦜️🔗 LangChain', 'description': 'New to LangChain or LLM app development in general? Read this material to quickly get up and running building your first applications.', 'language': 'en'}, page_content='Skip to main contentThese docs will be deprecated and no longer maintained with the release of LangChain v1.0 in October 2025. Visit the v1.0 alpha docsIntegrationsAPI ReferenceMoreContributingPeopleError referenceLangSmithLangGraphLangChain HubLangChain JS/TSv0.3v0.3v0.2v0.1💬SearchIntroductionTutorialsBuild a Question Answering application over a Graph DatabaseTut

In [6]:
## Converting chunks to vectors (Using vector embeddings (Openai embeddings))
from langchain_openai import OpenAIEmbeddings
embeddings=OpenAIEmbeddings()




In [7]:
#Store to vectorstore DB

from langchain_community.vectorstores import FAISS
vectorstoredb=FAISS.from_documents(documents, embeddings)

In [8]:
vectorstoredb

In [ ]:
#query and using similarity search , How we can query from VectorStore DB (where we stored our DB) and get response from it.
query="What will trace, monitor and evaluate your LLM application ?."
result=vectorstoredb.similarity_search(query)
result[0].page_content

'LangSmith\u200b\nLangSmith allows you to closely trace, monitor and evaluate your LLM application.\nIt seamlessly integrates with LangChain, and you can use it to inspect and debug individual steps of your chains as you build.\nLangSmith documentation is hosted on a separate site.\nYou can peruse LangSmith tutorials here.\nEvaluation\u200b\nLangSmith helps you evaluate the performance of your LLM applications. The tutorial below is a great way to get started:\n\nEvaluate your LLM application\nEdit this pagePreviousIntroductionNextBuild a Question Answering application over a Graph DatabaseGet startedOrchestrationLangSmithEvaluationCommunityLangChain ForumTwitterSlackGitHubOrganizationPythonJS/TSMoreHomepageBlogYouTubeCopyright © 2025 LangChain, Inc.'

In [11]:
from langchain_openai import ChatOpenAI
#create llm use api key
#eg, llm=ChatOpen(api_key=""), As I already used API key in env variable so no need to use it here. 
#How to call Specific model from openai
llm=ChatOpenAI(model="gpt-4o")

In [12]:
# Reterieval chain (Ask context/meaningfull question). / Document chain

#Retrieval chain is a set of modular components and orchestrated steps that allows a Large Language Model (LLM) to access, retrieve, and use information from an external knowledge base to generate more accurate and context-specific responses. 
#This process is the foundation of Retrieval-Augmented Generation (RAG).
#Very important

#https://vijaykumarkartha.medium.com/beginners-guide-to-retrieval-chain-from-langchain-f307b1a20e77


from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

#Create own custome prompt 
prompt=ChatPromptTemplate.from_template(
#(what's context ??? Information that I will be giving to LLM models regarding documents or text)
    """
Answer the following question based only on the provided context 
<context>
{context}
</context>
    """
    
)

# document chain >> will be responsible provding prompt for specific context.
## Give LLM + Propmt
document_chain=create_stuff_documents_chain(llm,prompt)

document_chain





RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableLambda(format_docs)
}), kwargs={}, config={'run_name': 'format_inputs'}, config_factories=[])
| ChatPromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template='\nAnswer the following question based only on the provided context \n<context>\n{context}\n</context>\n    '), additional_kwargs={})])
| ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x000001DE7D4DB190>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x000001DE7D70FB20>, root_client=<openai.OpenAI object at 0x000001DE7D4D9750>, root_async_client=<openai.AsyncOpenAI object at 0x000001DE7D70F8E0>, model_name='gpt-4o', model_kwargs={}, openai_api_key=SecretStr('**********'))
| StrOutputParser(), kwargs={}, confi

In [14]:
from langchain_core.documents import Document
document_chain.invoke({
  "input":"What will trace, monitor and evaluate your LLM application ?",
  "context":[Document(page_content="What will trace, monitor and evaluate your LLM application ?, you can use it to inspect and debug individual steps of your chains as you build.")]  
})

'The tool or feature that will trace, monitor, and evaluate your LLM application, allowing you to inspect and debug individual steps of your chains as you build, is not explicitly named in the provided context.'

In [ ]:
#However, We want the documents to first come from the retriever we just set up. That way, we can use the retriever to dynamically select the most relevant documents
#and pass those in for a given questions

In [15]:
# Input ---- > Reteriever --- > vectorstoredb
vectorstoredb

In [17]:
retriever=vectorstoredb.as_retriever()
from langchain.chains import create_retrieval_chain
#combine 1, retriever + 2, document chain
#retriever=vectorstoredb (as retriever)
#document chain = repsonible for giving you with context information.
retrieval_chain=create_retrieval_chain(retriever,document_chain)

In [18]:
retrieval_chain

RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableBinding(bound=RunnableLambda(lambda x: x['input'])
           | VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x000001DE7B962410>, search_kwargs={}), kwargs={}, config={'run_name': 'retrieve_documents'}, config_factories=[])
})
| RunnableAssign(mapper={
    answer: RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
              context: RunnableLambda(format_docs)
            }), kwargs={}, config={'run_name': 'format_inputs'}, config_factories=[])
            | ChatPromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template='\nAnswer the following question based only on the provided context \n<context>\n{context}\n</context>\n    '), additional_kwargs={})])
            | 

In [19]:
# get the response from the LLMs
response=retrieval_chain.invoke({"input":"What will trace, monitor and evaluate your LLM application ?"})
response["answer"]

'LangSmith is a tool designed to help you closely trace, monitor, and evaluate your LLM (Large Language Model) applications. It integrates with LangChain, allowing you to inspect and debug individual steps of your application chains as you build them. LangSmith is useful for evaluating the performance of LLM applications and offers tutorials to help you get started with such evaluations.'

In [20]:
response

{'input': 'What will trace, monitor and evaluate your LLM application ?',
 'context': [Document(id='aae736f8-ca58-4da6-bcdc-490e27c9854e', metadata={'source': 'https://python.langchain.com/docs/tutorials/', 'title': 'Tutorials | 🦜️🔗 LangChain', 'description': 'New to LangChain or LLM app development in general? Read this material to quickly get up and running building your first applications.', 'language': 'en'}, page_content='LangSmith\u200b\nLangSmith allows you to closely trace, monitor and evaluate your LLM application.\nIt seamlessly integrates with LangChain, and you can use it to inspect and debug individual steps of your chains as you build.\nLangSmith documentation is hosted on a separate site.\nYou can peruse LangSmith tutorials here.\nEvaluation\u200b\nLangSmith helps you evaluate the performance of your LLM applications. The tutorial below is a great way to get started:\n\nEvaluate your LLM application\nEdit this pagePreviousIntroductionNextBuild a Question Answering applic